In [ ]:
!pip install huggingface_hub boto3 minio

In [ ]:
# ====== Create minio client ======
from minio import Minio
from minio.error import S3Error
import  os

In [ ]:
# ====== CONFIGURE ======
MINIO_ENDPOINT = "SET BUCKET ENDPOINT" # Set your minio ENDPOINT
MINIO_ACSSESS_KEY = "minio" # Set your minio user name
MINIO_SECRET_KEY = "minio123" # Set you minio password

In [ ]:
# Create the MinIO client
client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACSSESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False  # Set to True if you are using HTTPS
)

In [ ]:
# ====== Downlaod the model from huggingface and upload to minio ======
import tempfile
import boto3
from huggingface_hub import hf_hub_download, list_repo_files

In [ ]:
# ====== CONFIGURE ======
repo_id = "meta-llama/Llama-3.1-8B-Instruct"  # Change to your model repo
revision = "main"
bucket_name = "your-bucket" # Your bucket name
huggingface_token = None # IF NEEDED ADD YOUR HUGGINGFACE TOKEN HERE

In [ ]:
# ====== DISABLE HF CACHE ======
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '0'
os.environ['HF_HOME'] = tempfile.mkdtemp()

In [ ]:
files = list_repo_files(repo_id=repo_id, revision=revision, token=huggingface_token)

for file in files:
    print(f"Processing: {file}")
    try:
        # 1. Download to a temp file
        temp_path = hf_hub_download(
            repo_id=repo_id,
            filename=file,
            revision=revision,
            cache_dir=os.environ['HF_HOME'],
            local_dir_use_symlinks=False,
        )

        # 2. Upload to MinIO
        try:
            client.fput_object(bucket_name, file, temp_path)
            print(f"'{file}' is successfully uploaded as object to bucket '{bucket_name}'.")
        except S3Error as e:
            print("Error occurred: ", e)

        # 3. Delete file to free memory
        os.remove(temp_path)
        print(f"Deleted local file: {temp_path}")

    except Exception as e:
        print(f"Error processing {file}: {e}")